### Install the packages

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
# store the url
url = "https://mtu.libanswers.com/"
resp = requests.get(url)
content = resp.text
soup = BeautifulSoup(content, 'html.parser')


In [27]:
# find all faqs
faqs = soup.find_all('div', class_='s-la-faq-listing-q')
# get the text
# store faqs and links
faqs_names = []
link_answers = []
for i in faqs:
    faq = i.get_text()
    faqs_names.append(faq)
    # link to the faq answer
    link = i.find('a')['href']
    link_answers.append(link)
     # Finding the <ul> element containing the topics
    topics_ul = soup.find('ul', class_='list-inline', attrs={'aria-label': 'Topics:'})
    print(topics_ul.prettify())
    # Extracting topics
    # topics = [topic.text for topic in topics_ul.find_all('a')]
    # print(topics)







<ul aria-label="Topics:" class="list-inline metavalue">
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=3D%20Printing">
    3D Printing
   </a>
  </span>
 </li>
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=Circulation">
    Circulation
   </a>
  </span>
 </li>
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=Facility">
    Facility
   </a>
  </span>
 </li>
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=Interlibrary%20Loan%20%28ILL%29">
    Interlibrary Loan (ILL)
   </a>
  </span>
 </li>
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=IT-Help">
    IT-Help
   </a>
  </span>
 </li>
 <li>
  <span class="label label-topic">
   <a href="/search/?t=0&amp;adv=1&amp;topics=Library%20Building">
    Library Building
   </a>
  </span>
 </li>
</ul>

<ul aria-label="Topics:" class="list-inline metavalue